In [ ]:
# imports

import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from multiprocessing import Pool
import multiprocessing
import threading
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Pool
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import validation_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
import matplotlib
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
from sklearn.cluster import KMeans
from scipy.stats import mode
from sklearn.decomposition import FastICA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import random_projection

Flag_2Dplot=1
Flag_3Dplot=1
num_red_Feat=10
n_ExtraTrees=40

In [ ]:
#---------- read data -  for classification

random_state=np.random.randint(low = 0, high = 100)
        
# in this section we read scsvfile and import dataset
df=pd.read_csv("ComputerSpeedData.csv")

# here we calculate average of 4 run time
df["RunF"]=(df["Run1 (ms)"]+df["Run2 (ms)"]+df["Run3 (ms)"]+df["Run4 (ms)"])/4

# here we drop run time columns and insert the average amount into a new column
df_clas=df.drop(columns=['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)'])
df_clas=df_clas.drop(columns=['RunF'])

# categorize based in mean
avg=df['RunF'].mean()
df['RunF_binary']=np.where(df['RunF']>=avg,1,0)
df_y=pd.DataFrame(df['RunF_binary'])

df=df.drop(columns=['RunF','RunF_binary'])

scaler = StandardScaler()
X=df_clas.values
X=scaler.fit_transform(X)
y=df_y.values

In [ ]:
#           Feature importance by Trees
forest = ExtraTreesClassifier(n_estimators=30, random_state=0)

forest.fit(X,y)

importances = forest.feature_importances_
a=(forest.estimators_)
print('\n a[0].feature_importances_ is: ', a[0].feature_importances_)

std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)

indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

In [ ]:
# Plot the feature importances of the forest by Feature Names
attr_labels=df_clas.columns
plt.figure()
plt.title("Feature importances by Feature Names")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), attr_labels, rotation=70)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
# Plot the feature importances of the forest by Feature Indices
plt.figure()
plt.title("Feature importances by Feature Indices")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices, rotation=70)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
#---------- read data -  for clustering
random_state=np.random.randint(low = 0, high = 100)
        
# in this section we read scsvfile and import dataset
df=pd.read_csv("ComputerSpeedData.csv")

# here we calculate average of 4 run time
df["RunF"]=(df["Run1 (ms)"]+df["Run2 (ms)"]+df["Run3 (ms)"]+df["Run4 (ms)"])/4

# here we drop run time columns and insert the average amount into a new column
df_clus=df.drop(columns=['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)'])
scaler = StandardScaler()
X_Clus=scaler.fit_transform(df_clus)

In [ ]:
#       dimension reduced data

pca = PCA(n_components=num_red_Feat)
X_pca=np.array(pca.fit_transform(X_Clus))
print(X_pca.shape)

ica = FastICA(n_components=num_red_Feat, random_state=0)
X_ica = ica.fit_transform(X_Clus)

rp = random_projection.GaussianRandomProjection(n_components=num_red_Feat)
X_rp = rp.fit_transform(X_Clus)

In [ ]:
# X_PCA feature importance
forest = ExtraTreesClassifier(n_estimators=n_ExtraTrees, random_state=0)
forest.fit(X_pca,y)
importances = forest.feature_importances_
a=(forest.estimators_)
print('\n a[0].feature_importances_ is: ', a[0].feature_importances_)

std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices_PCA = np.argsort(importances)[::-1]

In [ ]:
# Print the feature ranking
print("Feature ranking:")
for f in range(X_pca.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices_PCA[f], importances[indices_PCA[f]]))


In [ ]:
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances of X_pca data")
plt.bar(range(X_pca.shape[1]), importances[indices_PCA],
       color="r", yerr=std[indices_PCA], align="center")
plt.xticks(range(X_pca.shape[1]), indices, rotation=70)
plt.xlim([-1, X_pca.shape[1]])
plt.show()

In [ ]:
# X_ICA feature importance
forest = ExtraTreesClassifier(n_estimators=30, random_state=0)
forest.fit(X_ica,y)
importances = forest.feature_importances_
a=(forest.estimators_)
print('\n a[0].feature_importances_ is: ', a[0].feature_importances_)

std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices_ICA = np.argsort(importances)[::-1]

In [ ]:
# Print the feature ranking
print("Feature ranking:")
for f in range(X_ica.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices_ICA[f], importances[indices_ICA[f]]))

In [ ]:
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances of X_ica data")
plt.bar(range(X_ica.shape[1]), importances[indices_ICA],
       color="r", yerr=std[indices_ICA], align="center")
plt.xticks(range(X_ica.shape[1]), indices_ICA, rotation=70)
plt.xlim([-1, X_ica.shape[1]])
plt.show()


In [ ]:
# X_rp feature importance
forest = ExtraTreesClassifier(n_estimators=30, random_state=0)
forest.fit(X_rp,y)
importances = forest.feature_importances_
a=(forest.estimators_)
print('\n a[0].feature_importances_ is: ', a[0].feature_importances_)

std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices_RP = np.argsort(importances)[::-1]

In [ ]:
# Print the feature ranking
print("Feature ranking:")
for f in range(X_rp.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices_RP[f], importances[indices_RP[f]]))


In [ ]:
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances of X_rp data")
plt.bar(range(X_rp.shape[1]), importances[indices_RP],
       color="r", yerr=std[indices_RP], align="center")
plt.xticks(range(X_rp.shape[1]), indices_RP, rotation=70)
plt.xlim([-1, X_rp.shape[1]])
plt.show()

In [ ]:
#         KMeans of original data
kmeans=KMeans(n_clusters=2)
clusters=kmeans.fit_predict(X_Clus)

acc=accuracy_score(y,clusters)
if acc<0.5: acc=1-acc
print('Accuracy of KMeans of original data is: ',acc)

In [ ]:
if Flag_2Dplot:
    # plot original classification
    plt.figure()
    plt.scatter(X_Clus[:,0],X_Clus[:,1], c=y, s=40, cmap='RdYlBu')
    plt.title('Real Classification')
    plt.xlabel('MWG')
    plt.ylabel('NWG')
    
    # plot kmeans clustering
    plt.figure()
    plt.scatter(X_Clus[:,0],X_Clus[:,1], c=clusters, s=40, cmap='RdYlBu')
    plt.title('KMeans Clusters of original data')
    plt.xlabel('MWG')
    plt.ylabel('NWG')

In [ ]:
# plot 3D by 3 most important feature
if Flag_3Dplot:
    fig = plt.figure(figsize=(10, 10))
    ax = Axes3D(fig)
    ax.scatter(X_Clus[:, 0], X_Clus[:, 1], X_Clus[:, 4],
                c=clusters.astype(np.float),  
                edgecolor=None)
    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel('Feat 0')
    ax.set_ylabel('Feat 1')
    ax.set_zlabel('Feat 4')
    ax.set_title('KMeans Clusters of original data')
    ax.dist = 12

In [ ]:
# EM on original data
from sklearn.mixture import GaussianMixture

# covariance_type{‘full’ (default), ‘tied’, ‘diag’, ‘spherical’}
model=GaussianMixture(n_components=2,
              covariance_type= 'diag' , max_iter=100, random_state=0)

clusters=model.fit_predict(X_Clus)

acc=accuracy_score(y,clusters)
if acc<0.5: acc=1-acc
print('Accuracy of EM on original data is: ',acc)

In [ ]:
# plot kmeaans clustering
if Flag_2Dplot:
    plt.figure()
    plt.scatter(X_Clus[:,0],X_Clus[:,1], c=clusters, s=40, cmap='RdYlBu')
    plt.title('EM Clusters of original data')
    plt.xlabel('MWG')
    plt.ylabel('NWG')

In [ ]:
# plot 3D by 3 most important feature of originaal data
if Flag_3Dplot:
    fig = plt.figure(figsize=(10, 10))
    ax = Axes3D(fig)
    ax.scatter(X_Clus[:, 0], X_Clus[:, 1], X_Clus[:, 4],
                c=clusters.astype(np.float),  
                edgecolor=None)
    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel('Feat 0')
    ax.set_ylabel('Feat 1')
    ax.set_zlabel('Feat 4')
    ax.set_title('EM Clusters of original data')
    ax.dist = 12


In [ ]:
acc_KMeans_PCA = list()
acc_KMeans_ICA = list()
acc_KMeans_rp = list()

sse_KMeans_PCA = list()
sse_KMeans_ICA = list()
sse_KMeans_rp = list()

acc_EM_PCA = list()
acc_EM_ICA = list()
acc_EM_rp = list()

mse_EM_PCA = list()
mse_EM_ICA = list()
mse_EM_rp = list()


n_list=[14,13,12,11,10,9,8,7,6,5,4]
for k in n_list:
    print('\n========iteration no is: ========= \n', k)
    
    pca = PCA(n_components=k)
    X_pca=np.array(pca.fit_transform(X_Clus))
    
    ica = FastICA(n_components=k, random_state=0)
    X_ica = ica.fit_transform(X_Clus)
    
    rp = random_projection.GaussianRandomProjection(n_components=k)
    X_rp = rp.fit_transform(X_Clus)
    
    # extract feature importance of each reduced dataset
    # X_PCA feature importance
    forest = ExtraTreesClassifier(n_estimators=n_ExtraTrees, random_state=0)
    forest.fit(X_pca,y)
    importances = forest.feature_importances_
    
    indices_PCA = np.argsort(importances)[::-1]
    
   
    
    
    
    # X_ICA feature importance
    forest = ExtraTreesClassifier(n_estimators=30, random_state=0)
    forest.fit(X_ica,y)
    importances = forest.feature_importances_
    
    indices_ICA = np.argsort(importances)[::-1]
    
   
    
    
    # X_rp feature importance
    forest = ExtraTreesClassifier(n_estimators=30, random_state=0)
    forest.fit(X_rp,y)
    importances = forest.feature_importances_
   
    indices_RP = np.argsort(importances)[::-1]
    
   
    
    #%%         KMeans of PCA data
    kmeans=KMeans(n_clusters=2)
    clusters=kmeans.fit_predict(X_pca)
    sse_KMeans_PCA.append( kmeans.inertia_)
    acc=accuracy_score(y,clusters)
    
    if acc<0.5: acc=1-acc
    acc_KMeans_PCA.append(acc)
    print('Accuracy of KMeans on PCA data is: ',acc)
    
    # plot kmeans clustering
    if Flag_2Dplot:
        if k==4:
            plt.figure()
            plt.scatter(X_pca[:,indices_PCA[0]],X_pca[:,indices_PCA[1]],
                        c=clusters, s=40, cmap='RdYlBu')
            plt.title('KMeans Clusters of PCA data')
            plt.xlabel('Feat {}'.format(indices_PCA[0]))
            plt.ylabel('Feat {}'.format(indices_PCA[1]))
    
    
    
    # plot 3D by 3 most important feature of X_pca
    if Flag_3Dplot:
        if k==4:
            fig = plt.figure(figsize=(10, 10))
            ax = Axes3D(fig)
            ax.scatter(X_pca[:, 0], X_pca[:, 3], X_pca[:, 2],
                        c=clusters.astype(np.float),  
                        edgecolor=None)
            ax.w_xaxis.set_ticklabels([])
            ax.w_yaxis.set_ticklabels([])
            ax.w_zaxis.set_ticklabels([])
            ax.set_xlabel('Feat {}'.format(indices_PCA[0]))
            ax.set_ylabel('Feat {}'.format(indices_PCA[1]))
            ax.set_zlabel('Feat {}'.format(indices_PCA[2]))
            ax.set_title('KMeans Clusters of X_pca data')
            ax.dist = 12
    
    
    #%%
    
    
    
    
    #%%         KMeans of ICA data
    kmeans=KMeans(n_clusters=2)
    clusters=kmeans.fit_predict(X_ica)
    sse_KMeans_ICA.append(  kmeans.inertia_)
    acc=accuracy_score(y,clusters)
    
    if acc<0.5: acc=1-acc
    acc_KMeans_ICA.append(acc)
    print('Accuracy of KMeans on ICA data is: ',acc)
    
    # plot kmeans clustering
    if Flag_2Dplot:
        if k==4:
            plt.figure()
            plt.scatter(X_ica[:,indices_ICA[0]],X_ica[:,indices_ICA[1]], 
                        c=clusters, s=40, cmap='RdYlBu')
            plt.title('KMeans Clusters of ICA data')
            plt.xlabel('Feat {}'.format(indices_ICA[0]))
            plt.ylabel('Feat {}'.format(indices_ICA[1]))
    
    
    # plot 3D by 3 most important feature of X_pca
    if Flag_3Dplot:
        if k==4:
            fig = plt.figure(figsize=(10, 10))
            ax = Axes3D(fig)
            ax.scatter(X_ica[:, indices_ICA[0]], X_ica[:, indices_ICA[1]],
                       X_ica[:, indices_ICA[2]], c=clusters.astype(np.float),  
                        edgecolor=None)
            ax.w_xaxis.set_ticklabels([])
            ax.w_yaxis.set_ticklabels([])
            ax.w_zaxis.set_ticklabels([])
            ax.set_xlabel('Feat {}'.format(indices_ICA[0]))
            ax.set_ylabel('Feat {}'.format(indices_ICA[1]))
            ax.set_zlabel('Feat {}'.format(indices_ICA[2]))
            ax.set_title('KMeans Clusters of X_ica data')
            ax.dist = 12
    
    #%%
    
    
    
    
    #%%         KMeans of rp data
    kmeans=KMeans(n_clusters=2)
    clusters=kmeans.fit_predict(X_rp)
    sse_KMeans_rp.append(  kmeans.inertia_)
    acc=accuracy_score(y,clusters)
    
    if acc<0.5: acc=1-acc
    acc_KMeans_rp.append(acc)
    print('Accuracy of KMeans on rp data is: ',acc)
    
    # plot kmeans clustering
    if Flag_2Dplot:
        if k==4:
            plt.figure()
            plt.scatter(X_rp[:,indices_ICA[0]],X_rp[:,indices_ICA[1]], 
                        c=clusters, s=40, cmap='RdYlBu')
            plt.title('KMeans Clusters of rp data')
            plt.xlabel('Feat {}'.format(indices_ICA[0]))
            plt.ylabel('Feat {}'.format(indices_ICA[1]))
    
    
    # plot 3D by 3 most important feature of X_pca
    if Flag_3Dplot:
        if k==4:
            fig = plt.figure(figsize=(10, 10))
            ax = Axes3D(fig)
            ax.scatter(X_rp[:, indices_ICA[0]], X_rp[:, indices_ICA[1]],
                       X_rp[:, indices_ICA[2]], c=clusters.astype(np.float),  
                        edgecolor=None)
            ax.w_xaxis.set_ticklabels([])
            ax.w_yaxis.set_ticklabels([])
            ax.w_zaxis.set_ticklabels([])
            ax.set_xlabel('Feat {}'.format(indices_ICA[0]))
            ax.set_ylabel('Feat {}'.format(indices_ICA[1]))
            ax.set_zlabel('Feat {}'.format(indices_ICA[2]))
            ax.set_title('KMeans Clusters of X_rp data')
            ax.dist = 12
    
    #%%
    
    
    
    
    
    
    #%%                    EM   on PCA data
    
    from sklearn.mixture import GaussianMixture
    
    # covariance_type{‘full’ (default), ‘tied’, ‘diag’, ‘spherical’}
    model=GaussianMixture(n_components=2,
                  covariance_type= 'diag' , max_iter=100, random_state=0)
    
    clusters=model.fit_predict(X_pca)
    covariances = model.covariances_
    mse_EM_PCA.append( np.diag(covariances))
    acc=accuracy_score(y,clusters)
    
    if acc<0.5: acc=1-acc
    acc_EM_PCA.append(acc)
    print('Accuracy of EM on PCA data is: ',acc)
    
    # plot kmeaans clustering
    if Flag_2Dplot:
        if k==4:
            plt.figure()
            plt.scatter(X_pca[:,indices_PCA[0]],X_pca[:,indices_PCA[1]],
                        c=clusters, s=40, cmap='RdYlBu')
            plt.title('EM Clusters of PCA data')
            plt.xlabel('Feat {}'.format(indices_PCA[0]))
            plt.ylabel('Feat {}'.format(indices_PCA[1]))
    
    
    # plot 3D by 3 most important feature of X_pca data
    if Flag_3Dplot:
        if k==4:
            fig = plt.figure(figsize=(10, 10))
            ax = Axes3D(fig)
            ax.scatter(X_pca[:, indices_PCA[0]], X_pca[:, indices_PCA[1]],
                       X_pca[:, indices_PCA[2]], c=clusters.astype(np.float),  
                        edgecolor=None)
            ax.w_xaxis.set_ticklabels([])
            ax.w_yaxis.set_ticklabels([])
            ax.w_zaxis.set_ticklabels([])
            ax.set_xlabel('Feat {}'.format(indices_PCA[0]))
            ax.set_ylabel('Feat {}'.format(indices_PCA[1]))
            ax.set_zlabel('Feat {}'.format(indices_PCA[2]))
            ax.set_title('EM Clusters of X_pca data')
            ax.dist = 12
    
    #%%
    
    
    
    
    #%%                    EM   on ICA data
    
    from sklearn.mixture import GaussianMixture
    
    # covariance_type{‘full’ (default), ‘tied’, ‘diag’, ‘spherical’}
    model=GaussianMixture(n_components=2,
                  covariance_type= 'diag' , max_iter=100, random_state=0)
    
    clusters=model.fit_predict(X_ica)
    covariances = model.covariances_
    mse_EM_ICA.append( np.diag(covariances))
    acc=accuracy_score(y,clusters)
    
    if acc<0.5: acc=1-acc
    acc_EM_ICA.append(acc)
    print('Accuracy of EM on ICA data is: ',acc)
    
    # plot kmeaans clustering
    if Flag_2Dplot:
        if k==4:
            plt.figure()
            plt.scatter(X_ica[:,indices_ICA[0]],X_ica[:,indices_ICA[1]],
                        c=clusters, s=40, cmap='RdYlBu')
            plt.title('EM Clusters of ICA data')
            plt.xlabel('Feat {}'.format(indices_ICA[0]))
            plt.ylabel('Feat {}'.format(indices_ICA[1]))
    
    
    
    # plot 3D by 3 most important feature of X_ica data
    if Flag_3Dplot:
        if k==4:
            fig = plt.figure(figsize=(10, 10))
            ax = Axes3D(fig)
            ax.scatter(X_ica[:,indices_ICA[0]], X_ica[:, indices_ICA[1]],
                       X_ica[:, indices_ICA[2]], c=clusters.astype(np.float),  
                        edgecolor=None)
            ax.w_xaxis.set_ticklabels([])
            ax.w_yaxis.set_ticklabels([])
            ax.w_zaxis.set_ticklabels([])
            ax.set_xlabel('Feat {}'.format(indices_ICA[0]))
            ax.set_ylabel('Feat {}'.format(indices_ICA[1]))
            ax.set_zlabel('Feat {}'.format(indices_ICA[2]))
            ax.set_title('EM Clusters of X_ica data')
            ax.dist = 12
    
    #%%
    
    
    
    
    #%%                    EM   on rp data
    
    from sklearn.mixture import GaussianMixture
    
    # covariance_type{‘full’ (default), ‘tied’, ‘diag’, ‘spherical’}
    model=GaussianMixture(n_components=2,
                  covariance_type= 'diag' , max_iter=100, random_state=0)
    
    clusters=model.fit_predict(X_rp)
    covariances = model.covariances_
    mse_EM_rp.append(  np.diag(covariances))
    acc=accuracy_score(y,clusters)
    
    if acc<0.5: acc=1-acc
    acc_EM_rp.append(acc)
    print('Accuracy of EM on rp data is: ',acc)
    
    # plot kmeaans clustering
    if Flag_2Dplot:
        if k==4:
            plt.figure()
            plt.scatter(X_rp[:,indices_ICA[0]],X_rp[:,indices_ICA[1]], 
                        c=clusters, s=40, cmap='RdYlBu')
            plt.title('EM Clusters of X_rp data')
            plt.xlabel('Feat {}'.format(indices_ICA[0]))
            plt.ylabel('Feat {}'.format(indices_ICA[1]))
    
    
    # plot 3D by 3 most important feature of X_ica data
    if Flag_3Dplot:
        if k==4:
            fig = plt.figure(figsize=(10, 10))
            ax = Axes3D(fig)
            ax.scatter(X_rp[:, indices_ICA[0]], X_rp[:, indices_ICA[1]],
                       X_rp[:, indices_ICA[2]], c=clusters.astype(np.float),  
                        edgecolor=None)
            ax.w_xaxis.set_ticklabels([])
            ax.w_yaxis.set_ticklabels([])
            ax.w_zaxis.set_ticklabels([])
            ax.set_xlabel('Feat {}'.format(indices_ICA[0]))
            ax.set_ylabel('Feat {}'.format(indices_ICA[1]))
            ax.set_zlabel('Feat {}'.format(indices_ICA[2]))
            ax.set_title('EM Clusters of X_rp data')
            ax.dist = 12

In [ ]:
# plot SSE of KMeans        
plt.figure()
plt.plot(n_list,sse_KMeans_PCA,'b-o',label='pca')
plt.plot(n_list,sse_KMeans_ICA,'g-o',label='ica')
plt.plot(n_list,sse_KMeans_rp,'r-o',label='rp')
plt.legend(loc='best')
plt.title('SSE of KMeans for different dimensions')
plt.xlabel('n_components')
plt.ylabel('SSE')

In [ ]:
# plot acc of KMeans        
plt.figure()
plt.plot(n_list,acc_KMeans_PCA,'b-o',label='pca')
plt.plot(n_list,acc_KMeans_ICA,'g-o',label='ica')
plt.plot(n_list,acc_KMeans_rp,'r-o',label='rp')
plt.legend(loc='best')
plt.title('Accuracy of KMeans for different dimensions')
plt.xlabel('n_components')
plt.ylabel('Accuracy')

In [ ]:
# plot MSE of EM
mse_EM_PCA_1=list()
mse_EM_ICA_1=list()
mse_EM_rp_1=list()
for i in range(len(n_list)):
    mse_EM_PCA_1.append(mse_EM_PCA[i] [0])
    mse_EM_ICA_1.append(mse_EM_ICA[i] [0])
    mse_EM_rp_1.append(mse_EM_rp[i] [0])

plt.figure()
plt.plot(n_list,mse_EM_PCA_1,'b-o',label='pca')
plt.plot(n_list,mse_EM_ICA_1,'g-o',label='ica')
plt.plot(n_list,mse_EM_rp_1,'r-o',label='rp')
plt.legend(loc='best')
plt.title('MSE of EM for different dimensions')
plt.xlabel('n_components')
plt.ylabel('MSE')

In [ ]:
# plot Acc of EM
plt.figure()
plt.plot(n_list,acc_EM_PCA,'b-o',label='pca')
plt.plot(n_list,acc_EM_ICA,'g-o',label='ica')
plt.plot(n_list,acc_EM_rp,'r-o',label='rp')
plt.legend(loc='best')
plt.title('Accuracy of EM for different dimensions')
plt.xlabel('n_components')
plt.ylabel('Accuracy')